In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.126.25.242:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15179618586971532678),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11477313711438833736),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3909112506610504564),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2790264442861233248),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16423073202409392273),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5418269432589817399),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9205249566398100365),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 1838359373389712811),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 128524615648

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3A

In [0]:
ls

adc.json  drive/  sample_data/


In [0]:
cd drive/finalYrProject/Toxic_CC_1

In [0]:
ls

baselines/                  Model_load.ipynb
clean_data.ipynb            Model_load_modules.ipynb
en_ger_train_1.csv          model_stamp_check0.h5
en_ger_train_translate.csv  model_stamp_check1.h5
english_train_1.csv         model_stamp_check2.h5
english_train_2.csv         model_stamp_check3.h5
english_train_3.csv         model_stamp_check4.h5
english_train_4.csv         model_stamp_check5.h5
english_train_5.csv         model_stamp_check6.h5
english_train_6.csv         model_stamp_check7.h5
english_train_7.csv         model_stamp_check8.h5
english_train_8.csv         model_stamp_check9.h5
features/                   only_toxic.csv
kmax_text_cnn0.h5           predict_path/
kmax_text_cnn1.h5           Readme.md
kmax_text_cnn2.h5           results/
kmax_text_cnn3.h5           sotoxic/
kmax_text_cnn4.h5           tools/
kmax_text_cnn5.h5           train_bgru.ipynb
kmax_text_cnn6.h5           train_char_level.ipynb
kmax_text_cnn7.h5           train_cleaned_word_level.ipynb
kmax_text_cnn8.

In [0]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
import sys

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch


Using TensorFlow backend.


  HTTP error 403 while getting http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


In [0]:
import toxic_m.models.keras.model_set as model_set
from toxic_m.train import trainer
from toxic_m.data_helper.data_loader import DataLoader

In [0]:
path = 'Dataset/'
EMBEDDING_FILE='../Toxic_CC_2/features/crawl-300d-2M.vec'
#EMBEDDING_FILE='features/glove.840B.300d.txt'
#EMBEDDING_FILE='features/glove.twitter.27B.200d.txt'
TRAIN_DATA_FILE='../Toxic_CC_2/Dataset/cleaned_train.csv'
TEST_DATA_FILE='../Toxic_CC_2/Dataset/cleaned_test.csv'

MAX_SEQUENCE_LENGTH = 400
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300

In [0]:
train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

In [0]:
print(len(train_df))

159571


In [0]:

data_loader = DataLoader()
embeddings_index = data_loader.load_embedding(EMBEDDING_FILE)

Total 2000000 word vectors.


In [0]:
print(train_df)

                      id                                       comment_text
0       00001cee341fdb12  yo bitch ja rule is more succesful then you wi...
1       0000247867823ef7    = = from rfc = = the title is fine as it is imo
2       00013b17ad220c46         = = sources = = * zawe ashton on lapland —
3       00017563c3f7919a  : if you have a look back at the source the in...
4       00017695ad8997eb          i do not anonymously edit articles at all
5       0001ea8717f6de06  thank you for understanding i think very highl...
6       00024115d4cbde0f  please do not add nonsense to wikipedia such e...
7       000247e83dcc1211                   : dear god this site is horrible
8       00025358d4737918  only a fool can believe in such numbers the co...
9       00026d1092fe71cc  = = double redirects = = when fixing double re...
10      0002eadc3b301559  i think its crap that the link to roggenbier i...
11      0002f87b16116a7f  : : : somebody will invariably try to add reli...
12      0003

In [0]:
pwd

In [0]:
########################################
## process texts in datasets
########################################
print('Processing text dataset')
list_sentences_train = train_df["comment_text"].fillna("no comment").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("no comment").values

comments = []
for text in list_sentences_train:
    comments.append(text)
    
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text)

#tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='"#%&()+,-./:;<=>@[\\]^_`{|}~\t\n')
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Processing text dataset
Found 332116 unique tokens
Shape of data tensor: (159571, 400)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 400)


In [0]:
########################################
## prepare embeddings
########################################
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
#embedding_matrix = np.random.normal(loc=matrix_mean, scale=matrix_std, size=(nb_words, EMBEDDING_DIM))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
null_count = 0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        null_count += 1
print('Null word embeddings: %d' % null_count)

Preparing embedding matrix
Null word embeddings: 24105


## Training

In [0]:
def get_model():
    return model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)

keras_model_trainer = trainer.KerasModelTrainer(model_stamp='model_rnn_check_new', epoch_num=7, learning_rate=1e-3)

In [0]:
models, val_loss, total_auc, fold_predictions = keras_model_trainer.train_folds(data, y, fold_count=5, batch_size=256, get_model_func=get_model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 400, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidi

# New Section

In [0]:
print("Overall val-loss:", val_loss, "AUC", total_auc)

Overall val-loss: 0.03883709286218794 AUC 0.9897118668571803


In [0]:
ls

baselines/         kmax_text_cnn5.h5      Readme.md
clean_data.ipynb   kmax_text_cnn6.h5      results/
Dataset/           kmax_text_cnn7.h5      sotoxic/
features/          kmax_text_cnn8.h5      tools/
kmax_text_cnn0.h5  kmax_text_cnn9.h5      train_bgru.ipynb
kmax_text_cnn1.h5  LICENSE                train_char_level.ipynb
kmax_text_cnn2.h5  model_stamp_check0.h5  train_cleaned_word_level.ipynb
kmax_text_cnn3.h5  model_stamp_check1.h5  train_meta_label.ipynb
kmax_text_cnn4.h5  predict_path/          train_rhn.ipynb


## Predections

In [0]:
train_fold_preditcions = np.concatenate(fold_predictions, axis=0)

In [0]:
from sklearn.metrics import roc_auc_score

In [0]:
training_auc = roc_auc_score(y[:], train_fold_preditcions)
print("Training AUC", training_auc)

Training AUC 0.9894365390082481


In [0]:
#test_data = test_df
CLASSES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
submit_path_prefix = "predict_path"

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict(test_data, batch_size=256, verbose=1)
    test_predicts_list.append(test_predicts)
    np.save("predict_path/", test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
submit_path = submit_path_prefix + "-L{:4f}-A{:4f}.csv".format(val_loss, total_auc)
test_predicts.to_csv(submit_path, index=False)

Predicting testing results...
153164/153164 [==============================] - 40s 263us/step


In [0]:
print("Predicting training results...")

train_ids = train_df["id"].values
train_ids = train_ids.reshape((len(train_ids), 1))

train_predicts = pd.DataFrame(data=train_fold_preditcions, columns=CLASSES)
train_predicts["id"] = train_ids
train_predicts = train_predicts[["id"] + CLASSES]
submit_path = submit_path_prefix + "-Train-L{:4f}-A{:4f}.csv".format(val_loss, training_auc)
train_predicts.to_csv(submit_path, index=False)

Predicting training results...
